In [35]:
import requests
import pandas as pd
import json
from dotenv import load_dotenv
import os
import time

In [39]:
api_key = {}
try:
    with open('api_key.json') as file:
        api_key = json.load(file)
except FileNotFoundError:
    print("Error: credentials.json file not found.")

{'x-rapidapi-key': '28f61182d84a56cbc1d5d835057ec1ce'}


In [40]:
url = "https://v1.basketball.api-sports.io/standings?league=12&season=2022-2023"

payload = {}
headers = {
    'x-rapidapi-key': api_key['x-rapidapi-key'],
    'x-rapidapi-host': 'v1.basketball.api-sports.io'
}

In [41]:
# The team_id_grab() function grabs the teams id,name,conference and conference position and stores it in a DataFrame

def team_id_grab():
    # request.request sends a request of a specified method to the url (in this case the method is GET)
    response = requests.request("GET", url, headers=headers, data=payload)

    # json.loads deserialises a JSON object to a standard python object (good for parsing)
    # also it turns nulls to a None python object
    standings = json.loads(response.text)

    # team_lvl (team level) grabs the section of the returned data that we need to parse through to collect 
    # info on the team
    team_lvl = standings['response'][0]
       
    # Creating a DataFrame to store the team API ids
    team_ids_df = pd.DataFrame(columns = ['Team', 'Team ID', 'Conference', 'Conference Position'])
     
    # Creating variables that hold the requests daily limit remaining (100 per day) 
    # and the per minute requests remaining (10 per minute)
    daily_calls_remaining = response.headers['x-ratelimit-requests-remaining']
    perMinute_calls_remaining = response.headers['X-RateLimit-Remaining']
    
    print(f'Daily calls remaining = {daily_calls_remaining}')
    print(f'Calls left per minute = {perMinute_calls_remaining}\n')

    df_row = 0

    for team in team_lvl:
        if team['group']['name'] == "Western Conference" or team['group']['name'] == "Eastern Conference":
            t_id = int(team['team']['id'])
            t_name = team['team']['name']
            t_conference = team['group']['name']
            t_position = str(team['position'])

            team_ids_df.loc[df_row] = [t_name, t_id, t_conference, t_position]
            df_row += 1
    return team_ids_df

team_ids_df = team_id_grab()
print(team_ids_df)


Daily calls remaining = 49
Calls left per minute = 9

                      Team  Team ID          Conference Conference Position
0           Denver Nuggets      139  Western Conference                   1
1        Memphis Grizzlies      146  Western Conference                   2
2         Sacramento Kings      157  Western Conference                   3
3             Phoenix Suns      155  Western Conference                   4
4    Golden State Warriors      141  Western Conference                   5
5     Los Angeles Clippers      144  Western Conference                   6
6     New Orleans Pelicans      150  Western Conference                   7
7       Los Angeles Lakers      145  Western Conference                   8
8   Minnesota Timberwolves      149  Western Conference                   9
9    Oklahoma City Thunder      152  Western Conference                  10
10        Dallas Mavericks      138  Western Conference                  11
11               Utah Jazz      16